# Current Testing Battery

In [1]:
import sys
import statistics
import numpy
from itertools import product
from timeit import default_timer as timer
from tqdm import tqdm
sys.path.append("..")
import gillespy2
from scipy import stats

The performance of this package can be significantly increased if you install Cython.


In [2]:
from gillespy2.basic_ssa_solver import BasicSSASolver
from gillespy2.ssa_c_solver import SSACSolver
from gillespy2.basic_root_solver import BasicRootSolver
from gillespy2.optimized_ssa_solver import OptimizedSSASolver
from gillespy2.basic_tau_leaping_solver import BasicTauLeapingSolver
from gillespy2.basic_tau_hybrid_solver import BasicTauHybridSolver
from gillespy2.example_models import *

In [3]:
modelList = [Example(), Trichloroethylene(), MichaelisMenten(), Schlogl()]

solverList = [BasicSSASolver, SSACSolver, BasicRootSolver, OptimizedSSASolver, OptimizedSSASolver(), BasicTauLeapingSolver, BasicTauHybridSolver]
timingList = []

precompile_solver = True


for model, solver in product(modelList, solverList):
    medianList = []
    exterior_stats = []
    if precompile_solver and (solver == SSACSolver or isinstance(solver, SSACSolver)): #precompile SSACSolver to save time in repated tests
        solver = SSACSolver(model)
    for i in tqdm(range(30), desc = f'Model: {model.name}, Solver: {solver.name}'):
        standard_results = model.run()
        start = timer()
        test_results = model.run(solver=solver)
        stop = timer()
        medianList.append(stop-start)
        interior_stats = []
        for species in standard_results[0].keys():
            if solver in [BasicSSASolver, BasicRootSolver]:
                interior_stats.append(stats.ks_2samp(standard_results[0][species], test_results[species])[0])
            else:
                interior_stats.append(stats.ks_2samp(standard_results[0][species], test_results[0][species])[0])
                
        exterior_stats.append(statistics.median(interior_stats))
    median = statistics.median(medianList)
    timingList.append([model, solver, median])

SyntaxError: invalid syntax (<ipython-input-3-ac38bab16d8e>, line 14)